<a href="https://colab.research.google.com/github/Goodbam/Changing-Naver-rising-searches/blob/master/crawling(12_4_10pm).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install lxml
!pip install parse

In [0]:
import requests                 # url request
from lxml.html import parse     # parsing the data as the html form
from io import StringIO         # String I/O module

THRESHOLD = 16 #limit to check whether they are relative or not
NOISE = 9

In [28]:
url = "https://www.naver.com"

# Get html source
text = requests.get(url).text

# parsing the data as the html form
text_source = StringIO(text)    # read the data as string
parsed = parse(text_source)     # source ->parsing the data as the html form 

#find root node
doc = parsed.getroot()

# doc.findall(".//TagName") => find the tag lists
uls = doc.findall('.//ul')

#Get the Rising searches
ul = uls[10]   
spans = ul.findall('.//span')

data_list= []  # to save all the keywords

cnt = 1
for data in spans:
  if cnt % 2 == 0:  
    print(int((cnt+1)/2),'->',data.text_content().strip())  # show the existing rising searches
  cnt +=1  
  data_list.append(data.text_content().strip()) #save each keyword
    


1 -> 백석역
2 -> 이일재
3 -> 조정민
4 -> 필수지방산
5 -> 이선정
6 -> 황지만
7 -> 류현진 등번호
8 -> 양잿물
9 -> 서인영
10 -> 라라츄 헤어쿠션
11 -> 정가은
12 -> 이상벽
13 -> lj
14 -> 오일러 항등식
15 -> 정재형
16 -> 백석
17 -> 처음학교로
18 -> 라이신
19 -> 식품안전나라
20 -> 트로트 조정민


In [29]:
path='https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=' #This is the url for surfing

matrix = [] #to count the score of relationship
label = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] #disjoint set

cnt1 = 0

for i in range(0, 20):
  word = data_list[1 + cnt1]
  search = path + word #url searching the keyword
  text = requests.get(search).text # no worry about encoding url
  
  row = []
  cnt2 = 0
  for j in range(0, 20):
    if cnt1 != cnt2: #no need to check when they are equivalent
      target = data_list[1 + cnt2];
      row.append(text.count(target) + 7*word.count(target) - NOISE) 
 
      if row[int(cnt2/2)] >= THRESHOLD: #IF they are relative,
        print(word, ' and ', target,)
        #the representive number should be minimized
        
        #여기 정규화를 해서 row의합으로 각 row의 값을 나누던지 해보자
        #그러려니, 전체적으로 점수 자체가 높은 것은 감안해서 평균을 다시 루트씌우고 곱하든지 고려해보자
        #A와 B 관련 , B C 관련 이면 A C는 관련이 있는건가? 이것도 감안
        
        if label[int(cnt1/2)] < label[int(cnt2/2)]:
          label[int(cnt2/2)] = label[int(cnt1/2)]  
        
        else:
          label[int(cnt1/2)] = label[int(cnt2/2)]
    
    else :
      row.append(0)
    
    cnt2 += 2
  
  matrix.append(row)  
  cnt1 += 2
  

백석역  and  백석
이선정  and  황지만
이선정  and  서인영
이선정  and  정가은
이선정  and  lj
황지만  and  이선정
서인영  and  이선정
서인영  and  정가은
정가은  and  이선정
정가은  and  서인영
lj  and  이선정
lj  and  황지만
백석  and  백석역
트로트 조정민  and  조정민


In [30]:
print('\n',label,'\n')

print(matrix)

size = [0]
size = size*20

for i in range(0,20):
  size[i] = sum(matrix[i])

print(size)
score =[0]
score = score*20

for i in range(0,20):
  score[label[i] - 1] += 1/(i+1); #label is 1~20 => needed to subtract 1


 [1, 2, 3, 4, 5, 5, 7, 8, 5, 10, 5, 12, 5, 14, 15, 1, 17, 18, 19, 3] 

[[0, 0, 9, 0, 2, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 180, 0, 0, 0, 0], [2, 0, 9, 0, 2, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 11, 0, 0, 0, 0], [2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 11, 0, 0, 0, 1], [2, 0, 9, 0, 2, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 11, 0, 0, 0, 0], [2, 0, 9, 0, 0, 23, 0, 0, 21, 0, 28, 0, 18, 0, 0, 11, 0, 0, 0, 0], [2, 0, 9, 0, 106, 0, 0, 0, 4, 0, 2, 0, 13, 0, 0, 11, 0, 0, 0, 0], [2, 0, 9, 0, 2, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 11, 0, 0, 0, 0], [2, 0, 9, 0, 2, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 11, 0, 0, 0, 0], [2, 0, 9, 0, 42, 0, 0, 0, 0, 0, 23, 0, 10, 0, 0, 11, 0, 0, 0, 0], [2, 0, 9, 0, 2, 0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 11, 0, 0, 0, 0], [2, 0, 9, 0, 42, 2, 0, 0, 26, 0, 0, 0, 8, 0, 0, 11, 0, 0, 0, 0], [2, 0, 9, 0, 2, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 11, 0, 0, 0, 0], [2, 0, 9, 0, 69, 32, 0, 0, 2, 0, 4, 0, 0, 0, 0, 11, 0, 0, 0, 0], [2, 0, 9, 0, 2, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 11, 0, 0, 0, 0], [2, 0, 9, 0, 2, 0, 0, 0, 0, 0

In [31]:
#RESULT
print('\n☆급상승 검색어 순위☆\n')
cnt = 1;

#sorting according to SCORE
for i in range(0,20):
  max = 0
  for j in range(1,20):
    if score[j] > score[max]:
      max = j;
  
  relative_word = ''
  
  for j in range(0,20):
    if label[j] == max + 1:
      relative_word += data_list[1 + 2*j] + ' | ' 
  
  score[max] = 0;
  
  print(cnt, '. ', relative_word[0:-3])
  cnt += 1
  if(score.count(0) == 20):
    break  


☆급상승 검색어 순위☆

1 .  백석역 | 백석
2 .  이선정 | 황지만 | 서인영 | 정가은 | lj
3 .  이일재
4 .  조정민 | 트로트 조정민
5 .  필수지방산
6 .  류현진 등번호
7 .  양잿물
8 .  라라츄 헤어쿠션
9 .  이상벽
10 .  오일러 항등식
11 .  정재형
12 .  처음학교로
13 .  라이신
14 .  식품안전나라
